In [1]:
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

#note: for graphviz, need to install it separately, and also install the pygraphviz package.
#also, for plotly need to sign up and set the api key. refer to plotly set up guide on their website

Task 3 Viz 1: Showing a line chart animation of the rankings of sons in a particular region.

Try 1: Using Plotly line chart animations

In [2]:
rankings_file_path = "../../data/Daily_Ranking.csv"

In [3]:
df = pd.read_csv(rankings_file_path)

In [4]:
#first create dictionary of region code to country name for simplicity later
all_region_codes = ['ec', 'fr', 'ar', 'fi', 'no', 'it', 'lt', 'ph', 'tw', 'nz', 'ee', 'tr', 'us', 'sv', 'cr', 'de', 'cl', 'jp', 'br', 'hn', 'gt', 'ch', 'hu', 'ca', 'pe', 'be', 'my', 'dk', 'bo', 'pl', 'at', 'pt', 'se', 'mx', 'pa', 'uy', 'is', 'es', 'cz', 'ie', 'nl', 'sk', 'co', 'sg', 'id', 'do', 'lu', 'gb', 'py', 'au', 'lv', 'gr', 'hk']
df_countries = pd.read_json("../../data/countries.json")
df_countries = df_countries.transpose()
country_names = {r:df_countries.loc[r.upper()]['name'] for r in all_region_codes}

In [12]:
print("Select Region to Show in the Animation")
region_selector = widgets.Dropdown(
    options=[country_names[i] for i in all_region_codes],
    value='United States',
    description='Region:')
display(region_selector)

Select Region to Show in the Animation


Dropdown(description='Region:', index=12, options=('Ecuador', 'France', 'Argentina', 'Finland', 'Norway', 'Italy', 'Lithuania', 'Philippines', 'Taiwan', 'New Zealand', 'Estonia', 'Turkey', 'United States', 'El Salvador', 'Costa Rica', 'Germany', 'Chile', 'Japan', 'Brazil', 'Honduras', 'Guatemala', 'Switzerland', 'Hungary', 'Canada', 'Peru', 'Belgium', 'Malaysia', 'Denmark', 'Bolivia', 'Poland', 'Austria', 'Portugal', 'Sweden', 'Mexico', 'Panama', 'Uruguay', 'Iceland', 'Spain', 'Czech Republic', 'Ireland', 'Netherlands', 'Slovakia', 'Colombia', 'Singapore', 'Indonesia', 'Dominican Republic', 'Luxembourg', 'United Kingdom', 'Paraguay', 'Australia', 'Latvia', 'Greece', 'Hong Kong'), value='United States')